111401549吳欣庭
Spaceship Titanic

一、關於比賽

#選擇
一開始選擇的是Store Sales Prediction
不過對於初入門的我來說
好像有點太超綱了
(詳情見底下的附錄)
最後選擇老師推薦的，較簡單的，跟自身能力符合的Spaceship Titanic

#資料概述
Spaceship Titanic給的資料跟Titanic Survival相近
只不過需要整理的資料多了很多欄
空白資料也是分散且不少的
所以處理起來相對複雜一些

#資料說明
PassengerId 乘客編碼(數字)
HomePlanet 離開地(文字)
CryoSleep 是否休眠(T/F)
Cabin 艙室(數字)
Destination 目的地(文字)
Age 年齡(數字)
VIP 特殊服務(T/F)
RoomService/FoodCourt/ShoppingMall/Spa/VRDeck 服務(價錢)
Name 姓名(文字)
Transported 傳送成功(T/F)

#目標
PassengerId 乘客編碼(數字)
Transported 傳送成功(T/F)

二、實作

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

導入train資料

In [ ]:
df = pd.read_csv("spaceship_train.csv")
df.head()
df.info()

將姓名欄刪除

In [ ]:
df.drop(['Name'],axis=1,inplace=True)
df.head()

觀察各個變數與Transported的關係

In [ ]:
sns.pairplot(df[['Transported','HomePlanet']],dropna=True)
sns.pairplot(df[['Transported','CryoSleep']],dropna=True)
sns.pairplot(df[['Transported','Cabin']],dropna=True)
sns.pairplot(df[['Transported','Destination']],dropna=True)
sns.pairplot(df[['Transported','Age']],dropna=True)
sns.pairplot(df[['Transported','VIP']],dropna=True)
sns.pairplot(df[['Transported','RoomService']],dropna=True)
sns.pairplot(df[['Transported','FoodCourt']],dropna=True)
sns.pairplot(df[['Transported','ShoppingMall']],dropna=True)
sns.pairplot(df[['Transported','Spa']],dropna=True)
sns.pairplot(df[['Transported','VRDeck']],dropna=True)

觀察數字類型數據與Transported的關係

In [ ]:
df.groupby('Transported').mean(numeric_only=True)

觀察空值和有沒有空值太多而可以直接刪掉的欄位
->基本上每個欄位都有200個上下的空值，所以都沒有刪掉

In [ ]:
df.isnull().sum()
len(df)
len(df)/2
df.isnull().sum()>len(df)/2

觀察每一個欄位中的數據分布
->除了艙室之外，文字部分都只有三個選項

In [ ]:
df['HomePlanet'].value_counts()
df['CryoSleep'].value_counts()
df['Cabin'].value_counts()
df['Destination'].value_counts()
df['Age'].value_counts()
df['VIP'].value_counts()
df['RoomService'].value_counts()
df['FoodCourt'].value_counts()
df['ShoppingMall'].value_counts()
df['Spa'].value_counts()
df['VRDeck'].value_counts()

處理數字欄位的空值
->年齡採用中位數
->其他服務的價錢採用平均值

In [ ]:
df['Age'].median()
df['Age'].fillna(df['Age'].median(),inplace=True)
df['RoomService'].mean()
df['RoomService'].fillna(df['RoomService'].mean(),inplace=True)
df['FoodCourt'].mean()
df['FoodCourt'].fillna(df['FoodCourt'].mean(),inplace=True)
df['ShoppingMall'].mean()
df['ShoppingMall'].fillna(df['ShoppingMall'].mean(),inplace=True)
df['Spa'].mean()
df['Spa'].fillna(df['Spa'].mean(),inplace=True)
df['VRDeck'].mean()
df['VRDeck'].fillna(df['VRDeck'].mean(),inplace=True)

處理文字欄為的空值
->採用出現次數最多的

In [ ]:
df['HomePlanet'].value_counts().idxmax()
df['HomePlanet'].fillna(df['HomePlanet'].value_counts().idxmax(),inplace=True)
df['CryoSleep'].value_counts().idxmax()
df['CryoSleep'].fillna(df['CryoSleep'].value_counts().idxmax(),inplace=True)
df['Destination'].value_counts().idxmax()
df['Destination'].fillna(df['Destination'].value_counts().idxmax(),inplace=True)
df['VIP'].value_counts().idxmax()
df['VIP'].fillna(df['VIP'].value_counts().idxmax(),inplace=True)

艙等欄位刪除
->艙室編碼繁雜且不重複，無法用出現次數最多的值填空值

In [ ]:
df.drop(['Cabin'],axis=1,inplace=True)

觀察空值是否解決

In [ ]:
df.isnull().sum()

將非數字的欄位進行格式轉換

In [ ]:
df=pd.get_dummies(data=df,dtype=int,columns=['HomePlanet','CryoSleep','Destination','VIP'])
df.head()

將Transported與其他資料區分

In [ ]:
df.corr()
x = df.drop(['Transported'],axis=1)
y = df['Transported']

分訓練和測試資料

In [ ]:
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.3,random_state=67)

訓練模型

In [ ]:
from sklearn.linear_model import LogisticRegression
lr=LogisticRegression(max_iter=200)
lr.fit(x_train,y_train)
predictions=lr.predict(x_test)

評估模型

In [ ]:
from sklearn.metrics import confusion_matrix,accuracy_score,recall_score,precision_score
accuracy_score(y_test,predictions)
recall_score(y_test,predictions)
precision_score(y_test,predictions)
pd.DataFrame(confusion_matrix(y_test,predictions),columns=['Predict not Transported','Predict Transported'],index=['True not Transported','True Transported'])

模型輸出

In [ ]:
import joblib
joblib.dump(lr,'spaceship_model.pkl',compress=3)

使用模型

In [ ]:
import joblib
model_pretained=joblib.load('spaceship_model.pkl')

導入測試資料

In [ ]:
import pandas as pd
df_test=pd.read_csv("spaceship_test.csv")

把姓名和艙室欄位去掉

In [ ]:
df_test.drop(['Name'],axis=1,inplace=True)
df_test.drop(['Cabin'],axis=1,inplace=True)

將測試資料轉成訓練資料(模型使用)的樣子

In [ ]:
df_test['Age'].median()
df_test['Age'].fillna(df_test['Age'].median(),inplace=True)
df_test['RoomService'].mean()
df_test['RoomService'].fillna(df_test['RoomService'].mean(),inplace=True)
df_test['FoodCourt'].mean()
df_test['FoodCourt'].fillna(df_test['FoodCourt'].mean(),inplace=True)
df_test['ShoppingMall'].mean()
df_test['ShoppingMall'].fillna(df_test['ShoppingMall'].mean(),inplace=True)
df_test['Spa'].mean()
df_test['Spa'].fillna(df_test['Spa'].mean(),inplace=True)
df_test['VRDeck'].mean()
df_test['VRDeck'].fillna(df_test['VRDeck'].mean(),inplace=True)
df_test['HomePlanet'].value_counts().idxmax()
df_test['HomePlanet'].fillna(df_test['HomePlanet'].value_counts().idxmax(),inplace=True)
df_test['CryoSleep'].value_counts().idxmax()
df_test['CryoSleep'].fillna(df_test['CryoSleep'].value_counts().idxmax(),inplace=True)
df_test['Destination'].value_counts().idxmax()
df_test['Destination'].fillna(df_test['Destination'].value_counts().idxmax(),inplace=True)
df_test['VIP'].value_counts().idxmax()
df_test['VIP'].fillna(df_test['VIP'].value_counts().idxmax(),inplace=True)
df_test=pd.get_dummies(data=df_test,dtype=int,columns=['HomePlanet','CryoSleep','Destination','VIP'])

執行

In [ ]:
predictions2=model_pretained.predict(df_test)
predictions2

生成csv檔和匯出

In [ ]:
forSubmissionDF=pd.DataFrame(columns=['PassengerId','Transported'])
forSubmissionDF['PassengerId'] = df_test['PassengerId']
forSubmissionDF['Transported'] = predictions2
forSubmissionDF
forSubmissionDF.to_csv('spaceship_submission_1.csv',index=False)

三、過程

#資料清理
與Titanic Survive最大的不同是空值
->每一個欄位都有空值
->每一個欄位空值數目都差不多
->每一個欄位皆須進行空值處理

#小狀況
->艙室欄位由房號/編號/左右三個小元素組成，有獨特性，無法補齊空值
->最後生成csv檔的時候，PassengerId非單純數字，需抓出原test檔的該欄位

四、結果

#成績
0.78045
(比想像中高，好像比titanic survive高一點)

#改善
艙室資料直接刪除
->可以將三個小元素變成三個欄位分開分析
特殊服務的價錢採總平均值瑱入
->可以根據是否為VIP做出其他分類平均值填入
出發地與目的地的空值蔡出現次數最多的值填入
->可以依照Id裡的團體編號進行改動

附錄：一個自不量力的故事。。。

在進去Kaggle後
"Store Sales Prediction"直接進到我的眼睛
企管系的身分告訴我
就是他了！

進去看了資料以後
發現這個好！複！雜！
一般只會有一個train的檔案
他除了有train的檔案
還有油價，節假日，商店等等的檔案
甚至還有其他資訊要處理
不過我還是開始了

在上網看了許多類似(？)這題的解說
和花費了不少時間理解資料的內容後
我打開了python
再把全部資料都讀進去後
我迷茫了
要怎麼把兩個檔案的資料集合起來？
要怎麼把每個月的15號跟最後一天抓出來？
商店的各個資訊到底有甚麼意義？
......？
......？
......？

總而言之
在最後的最後
我認清了自己的能力
放棄了這個不屬於現階段的我的題目
用現有的能力
把Spaceship Titanic做到最好